In [ ]:
!pip install -q tensorflow-gpu==2.0.0-rc1

In [1]:
!pip install transformers

In [2]:
import os
import tensorflow as tf
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
base_path = '/content/gdrive/My Drive/Colab Notebooks'

In [31]:
data_in_path = base_path + '/data_in/'
data_out_path = base_path + '/data_out/'
train_path = 'ratings_train.txt'
test_path = 'ratings_test.txt'

train_data = pd.read_csv(data_in_path+train_path, delimiter='\t')
test_data = pd.read_csv(data_in_path+test_path, delimiter='\t')

In [10]:
from transformers import BertTokenizer, TFBertForSequenceClassification

In [33]:
model_name = 'bert-base-multilingual-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name) # 모델 학습 불러오기

I0314 14:02:03.959411 4391882176 filelock.py:274] Lock 140619432552096 acquired on /Users/chojunghyun/.cache/torch/transformers/bb773818882b0524dc53a1b31a2cc95bc489f000e7e19773ba07846011a6c711.535306b226c42cebebbc0dabc83b92ab11260e9919e21e2ab0beb301f267b4c7.lock
I0314 14:02:03.961528 4391882176 file_utils.py:413] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-vocab.txt not found in cache or force_download set to True, downloading to /Users/chojunghyun/.cache/torch/transformers/tmp91ndwxgt


I0314 14:02:06.127022 4391882176 file_utils.py:423] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-vocab.txt in cache at /Users/chojunghyun/.cache/torch/transformers/bb773818882b0524dc53a1b31a2cc95bc489f000e7e19773ba07846011a6c711.535306b226c42cebebbc0dabc83b92ab11260e9919e21e2ab0beb301f267b4c7
I0314 14:02:06.128233 4391882176 file_utils.py:426] creating metadata file for /Users/chojunghyun/.cache/torch/transformers/bb773818882b0524dc53a1b31a2cc95bc489f000e7e19773ba07846011a6c711.535306b226c42cebebbc0dabc83b92ab11260e9919e21e2ab0beb301f267b4c7
I0314 14:02:06.130275 4391882176 filelock.py:318] Lock 140619432552096 released on /Users/chojunghyun/.cache/torch/transformers/bb773818882b0524dc53a1b31a2cc95bc489f000e7e19773ba07846011a6c711.535306b226c42cebebbc0dabc83b92ab11260e9919e21e2ab0beb301f267b4c7.lock
I0314 14:02:06.131350 4391882176 tokenization_utils.py:418] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilin

I0314 14:02:07.164875 4391882176 filelock.py:274] Lock 140619424095368 acquired on /Users/chojunghyun/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.c7892120c5a9b21e515abc904e398dbabddf9510b122f659063cbf361fe16868.lock
I0314 14:02:07.166018 4391882176 file_utils.py:413] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json not found in cache or force_download set to True, downloading to /Users/chojunghyun/.cache/torch/transformers/tmp2o5_tk9x


I0314 14:02:08.119249 4391882176 file_utils.py:423] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-config.json in cache at /Users/chojunghyun/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.c7892120c5a9b21e515abc904e398dbabddf9510b122f659063cbf361fe16868
I0314 14:02:08.120486 4391882176 file_utils.py:426] creating metadata file for /Users/chojunghyun/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.c7892120c5a9b21e515abc904e398dbabddf9510b122f659063cbf361fe16868
I0314 14:02:08.121914 4391882176 filelock.py:318] Lock 140619424095368 released on /Users/chojunghyun/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.c7892120c5a9b21e515abc904e398dbabddf9510b122f659063cbf361fe16868.lock
I0314 14:02:08.122792 4391882176 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/b

I0314 14:02:09.148559 4391882176 filelock.py:274] Lock 140620825334000 acquired on /Users/chojunghyun/.cache/torch/transformers/7efc9507bca9e880aea7a38a849d8e16fcd54f2071f8f8143afa5815d00a16f4.25728a4fd7ddaafee2965f5821a206f237b83c672e0bb092881f9b1f5eea2b2f.h5.lock
I0314 14:02:09.149769 4391882176 file_utils.py:413] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-tf_model.h5 not found in cache or force_download set to True, downloading to /Users/chojunghyun/.cache/torch/transformers/tmp0giodbui


I0314 14:09:04.021545 4391882176 file_utils.py:423] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-uncased-tf_model.h5 in cache at /Users/chojunghyun/.cache/torch/transformers/7efc9507bca9e880aea7a38a849d8e16fcd54f2071f8f8143afa5815d00a16f4.25728a4fd7ddaafee2965f5821a206f237b83c672e0bb092881f9b1f5eea2b2f.h5
I0314 14:09:04.022713 4391882176 file_utils.py:426] creating metadata file for /Users/chojunghyun/.cache/torch/transformers/7efc9507bca9e880aea7a38a849d8e16fcd54f2071f8f8143afa5815d00a16f4.25728a4fd7ddaafee2965f5821a206f237b83c672e0bb092881f9b1f5eea2b2f.h5
I0314 14:09:04.024003 4391882176 filelock.py:318] Lock 140620825334000 released on /Users/chojunghyun/.cache/torch/transformers/7efc9507bca9e880aea7a38a849d8e16fcd54f2071f8f8143afa5815d00a16f4.25728a4fd7ddaafee2965f5821a206f237b83c672e0bb092881f9b1f5eea2b2f.h5.lock
I0314 14:09:04.024685 4391882176 modeling_tf_utils.py:338] loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/

I0314 14:09:06.083389 4391882176 modeling_tf_utils.py:376] Layers of TFBertForSequenceClassification not initialized from pretrained model: ['dropout_37', 'classifier']
I0314 14:09:06.084089 4391882176 modeling_tf_utils.py:380] Layers from pretrained model not used in TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']


In [46]:
example_text = train_data.document.iloc[0]

encoded_example = tokenizer.encode(example_text)
decoded_example = tokenizer.decode(encoded_example)

print(f'original text\n{example_text}\n')
print(f'encoded text\n{encoded_example}\n')
print(f'decoded text\n{decoded_example}\n')

original text
아 더빙.. 진짜 짜증나네요 목소리

encoded text
[101, 1174, 25539, 23236, 29234, 13045, 119, 119, 87550, 97082, 25539, 1176, 25539, 24937, 13045, 16801, 72197, 47024, 1169, 70724, 22585, 13926, 102]

decoded text
[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]



In [47]:
tokenizer.vocab

OrderedDict([('[PAD]', 0),
             ('[unused1]', 1),
             ('[unused2]', 2),
             ('[unused3]', 3),
             ('[unused4]', 4),
             ('[unused5]', 5),
             ('[unused6]', 6),
             ('[unused7]', 7),
             ('[unused8]', 8),
             ('[unused9]', 9),
             ('[unused10]', 10),
             ('[unused11]', 11),
             ('[unused12]', 12),
             ('[unused13]', 13),
             ('[unused14]', 14),
             ('[unused15]', 15),
             ('[unused16]', 16),
             ('[unused17]', 17),
             ('[unused18]', 18),
             ('[unused19]', 19),
             ('[unused20]', 20),
             ('[unused21]', 21),
             ('[unused22]', 22),
             ('[unused23]', 23),
             ('[unused24]', 24),
             ('[unused25]', 25),
             ('[unused26]', 26),
             ('[unused27]', 27),
             ('[unused28]', 28),
             ('[unused29]', 29),
             ('[unused30]', 30),
 

In [77]:
train_encoded_texts = []
train_labels = []

for text, label in tqdm_notebook(train_data[['document','label']].values):
    try:
        encoded_text = tokenizer.encode(text)
    except:
        print(text)
        continue
    train_encoded_texts.append(encoded_text)
    train_labels.append(label)

nan
nan
nan
nan
nan



In [57]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [79]:
train_padded_text = pad_sequences(train_encoded_texts, 50)
train_labels = np.array(train_labels)

In [80]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[metric])

In [81]:
checkpoint_path = data_out_path + model_name + '/weights'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True)

model.fit(x=train_padded_text,
          y=train_labels,
          batch_size=64,
          validation_split=0.1,
          callbacks=[cp_callback])

Train on 134995 samples, validate on 15000 samples
   960/134995 [..............................] - ETA: 5:20:41 - loss: 0.7039 - accuracy: 0.5391
Epoch 00001: saving model to ./data_out/bert-base-multilingual-uncased/weights
   960/134995 [..............................] - ETA: 5:38:59 - loss: 0.7039 - accuracy: 0.5391

KeyboardInterrupt: 

In [78]:
test_encoded_texts = []
test_labels = []

for text, label in tqdm_notebook(test_data[['document','label']].values):
    try:
        encoded_text = tokenizer.encode(text)
    except:
        print(text)
        continue
    test_encoded_texts.append(encoded_text)
    test_labels.append(label)

nan
nan
nan



In [53]:
test_output = model.predict(test_encoded_texts)

In [54]:
test_output = np.array(test_output)

In [82]:
print(test_output == test_labels).mean()

0.8753
